In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings


warnings. filterwarnings('ignore')
pd.set_option('display.max_columns',None)


In [ ]:
credit = pd.read_csv('../input/tmdb-movie-metadata/tmdb_5000_credits.csv')
movies = pd.read_csv('../input/tmdb-movie-metadata/tmdb_5000_movies.csv') 

In [ ]:
credit.head()

In [ ]:
movies.head()

In [ ]:
print('Credit:',credit.shape)
print('Movie:',movies.shape)

In [ ]:
credit = credit.rename(index=str,columns={'movie_id':'id'})
credit.columns

In [ ]:
movies.columns

In [ ]:
###### merging the two datasets

movie_merge = pd.merge(credit,movies,on='id')
movie_merge.head()

In [ ]:
movie_merge.shape

In [ ]:
movie_merge.columns

In [ ]:
movie_merge['status'].unique()

In [ ]:
movie_merge_clean = movie_merge.drop(['title_x','homepage','production_countries','title_y','status'],axis=1)

In [ ]:
movie_merge_clean.info()

In [ ]:
C = movie_merge_clean['vote_average'].mean()

In [ ]:
v= movie_merge_clean['vote_count']
R= movie_merge_clean['vote_average']
C= C
m = movie_merge_clean['vote_count'].quantile(.7)

In [ ]:
movie_merge_clean['weighted_average'] = ((R*v)+(C*m))/(v+m)

In [ ]:
movie_merge_clean.head()

In [ ]:
movie_rank = movie_merge_clean.sort_values('weighted_average',ascending = False)
movie_rank.head(10)

In [ ]:
movie_merge_clean.columns

In [ ]:
movie_rank[['id','original_language','original_title','popularity','vote_average','vote_count','weighted_average']].head(10)

# Best movie by average weights

In [ ]:
wa = movie_rank.sort_values('weighted_average',ascending=False)
plt.figure(figsize=(10,8))
sns.barplot(y=wa['original_title'].head(15),x=wa['weighted_average'].head(15))
plt.title('Best movie by average weights')

# Best movie by Popularity

In [ ]:
pop = movie_rank.sort_values('popularity',ascending=False)
plt.figure(figsize=(10,8))
sns.barplot(y=pop['original_title'].head(15),x=pop['popularity'].head(15))
plt.title('Best movie by Popularity')

### Recommendation for both Weighted Average and Popularity (50% - 50%)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaled_movies= MinMaxScaler().fit_transform(movie_merge_clean[['weighted_average','popularity']])
normalised_movie = pd.DataFrame(scaled_movies,columns=['Weighted Average','Popularity'])
normalised_movie

In [ ]:
movie_rank[['Normalised Weights','Normalised Population']] = normalised_movie
movie_rank

In [ ]:
movie_rank['Score'] = movie_rank['Normalised Weights']*.5 + movie_rank['Normalised Population']*.5
movie_rank = movie_rank.sort_values(['Score'],ascending=False)
movie_rank

In [ ]:
movie_rank[['original_title','weighted_average','vote_count','vote_average','Score','Normalised Weights','Normalised Population']].sort_values('Score',ascending=False)

# Top 10 movies based on Popularity and Weighted Average

In [ ]:
Score = movie_rank.sort_values('Score',ascending=False)
plt.figure(figsize=(10,8))
sns.barplot(y=Score['original_title'].head(10),x=Score['Score'].head(10))
plt.title('Best movie by Score')